This notebook prepares the data for the subsequent notebook `10-Step-Analyze.ipynb`, which generates figures illustrating the conservation properties of the heat equation with Neumann boundary conditions (with a=0, b=1), as described in Supplementary Material Section 3.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
num_devices = torch.cuda.device_count()
print("Number of visible GPUs:", num_devices)

for i in range(num_devices):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

current_device = torch.cuda.current_device()
print("Current device index:", current_device)
print("Current device name:", torch.cuda.get_device_name(current_device))

import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

from tqdm import tqdm
from data_processing import (
    SimpleSerializerSettings, scale_2d_array, unscale_2d_array,
    serialize_2d_integers, deserialize_2d_integers, extract_training_and_test
)
from heat_equation import (
    compute_exact_solution_random_ic_vary_Nx,
    visualize_spline_ic,
    solve_heat_ftcs, solve_heat_btcs,
)

from llama_utils import load_model_and_tokenizer, generate_text_multiple

MODEL_NAME = "meta-llama/Llama-3.1-8B"
# MODEL_NAME = "meta-llama/Llama-3.2-3B"
# MODEL_NAME = "meta-llama/Llama-3.2-1B"

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [ ]:
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)

In [ ]:
# Example: Demonstrating the process of generating and visualizing a random initial condition
L = 2
Nx = 14
init_cond_random = np.random.uniform(0, 1, size=Nx)
fig, cs = visualize_spline_ic(L, Nx, init_cond_random)
plt.tight_layout()
plt.show()

In [ ]:
# Define parameters for the Heat equation solver
L = 2       # Length of the spatial domain
k = 0.01    # Thermal diffusivity
T = 0.5     # Total simulation time
Nt = 25     # Number of time steps
dt = T/Nt
Nx = 14     # Number of spatial steps (excluding boundary points)
dx = L/(Nx+1)
settings = SimpleSerializerSettings(space_sep=",", time_sep=";")
input_time_steps = 16
number_of_future_predictions = 10
n_ics = 20
n_runs_per_ic = 1

def compute_integral_with_boundaries(u_interior, Nx, L):
    """
    Compute integral of solution including estimated boundary values.
    """
    x_full = np.linspace(-L/2, L/2, Nx + 2)
    u_full = np.zeros(Nx + 2)
    u_full[1:-1] = u_interior
    # Estimate boundary values using Neumann BC (consistent with FD schemes)
    if Nx >= 2:
        u_full[0] = (4*u_interior[0] - u_interior[1]) / 3.0
        u_full[-1] = (4*u_interior[-1] - u_interior[-2]) / 3.0
    else:
        u_full[0] = u_full[-1] = u_interior[0]
    
    return np.trapezoid(u_full, x_full)


def compute_exact_solution_conservation(
    u_exact_all, Nx, L, initial_integral_fine
):
    """
    Compute conservation error for the high-accuracy exact solution
    evaluated on the coarse grid. This represents the best achievable
    conservation on the coarse grid.
    """
    integral_changes = []
    for t in range(u_exact_all.shape[0]):
        # Compute integral on coarse grid with boundary reconstruction
        coarse_integral = compute_integral_with_boundaries(u_exact_all[t], Nx, L)
        # Compare against fine-grid initial integral
        relative_change = np.abs((coarse_integral - initial_integral_fine) / 
                                np.abs(initial_integral_fine) * 100)
        integral_changes.append(relative_change)
    
    return integral_changes


def log_ci(mean, std, n, tcrit):
    """
    95% CI for log10 axis
    mean : arithmetic mean of the n samples
    std : sample std of the n samples
    n : number of samples
    tcrit: two-sided t critical value
    """
    se = std / np.sqrt(n)  # SE in linear space
    se_log = se / (mean * np.log(10))  # delta-method SE in log space
    mean_log = np.log10(mean)
    delta_log = tcrit * se_log
    return 10**(mean_log - delta_log), 10**(mean_log + delta_log)


def linear_ci(mean, std, n, tcrit):
    """95% CI for linear scale"""
    se = std / np.sqrt(n)
    delta = tcrit * se
    return mean - delta, mean + delta


def llm_multi_predictions_with_conservation(
    full_serialized_data, input_time_steps, number_of_future_predictions,
    model, tokenizer, Nx, settings, vmin, vmax, n_seeds, initial_integral
):
    """LLM prediction function that also tracks energy conservation"""
    all_rows_scaled = deserialize_2d_integers(full_serialized_data, settings)
    all_seeds_integral_changes = []
    all_seeds_max_diffs = []
    all_seeds_rmses = []
    for seed in range(n_seeds):
        if n_seeds > 1:
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        # run a single LLM prediction sequence
        train_serial, _ = extract_training_and_test(full_serialized_data, input_time_steps, settings)
        if not train_serial.endswith(settings.time_sep):
            train_serial += settings.time_sep
        current_prompt = train_serial
        integral_changes = []
        max_diffs = []
        rmses = []
        for step_idx in range(number_of_future_predictions):
            gt_idx = input_time_steps + step_idx
            if gt_idx >= all_rows_scaled.shape[0]:
                # Stop if we exceed the available ground truth
                break
            next_line, _ = generate_text_multiple(current_prompt, model, tokenizer, Nx)
            next_line = next_line.strip()
            predicted_scaled_2d = deserialize_2d_integers(next_line, settings)
            predicted_unscaled_2d = unscale_2d_array(predicted_scaled_2d, vmin, vmax)
            if predicted_unscaled_2d.ndim == 2 and predicted_unscaled_2d.shape[0] == 1:
                pred_unscaled = predicted_unscaled_2d[0]
            else:
                pred_unscaled = predicted_unscaled_2d
            # Compute integral and relative change
            pred_integral = compute_integral_with_boundaries(pred_unscaled, Nx, L)
            relative_change = np.abs((pred_integral - initial_integral) / np.abs(initial_integral) * 100)
            integral_changes.append(relative_change)
            # Compute regular error metrics
            gt_scaled = all_rows_scaled[gt_idx]
            gt_unscaled = unscale_2d_array(gt_scaled[np.newaxis, :], vmin, vmax)[0]
            max_diff = np.max(np.abs(pred_unscaled - gt_unscaled))
            rmse = np.sqrt(np.mean((pred_unscaled - gt_unscaled)**2))
            max_diffs.append(max_diff)
            rmses.append(rmse)
            current_prompt += next_line + settings.time_sep
        all_seeds_integral_changes.append(integral_changes)
        all_seeds_max_diffs.append(max_diffs)
        all_seeds_rmses.append(rmses)
    max_steps = min(len(changes) for changes in all_seeds_integral_changes)
    avg_integral_changes = []
    std_integral_changes = []
    avg_max_diffs = []
    avg_rmses = []
    for step in range(max_steps):
        step_changes = [seed_changes[step] for seed_changes in all_seeds_integral_changes]
        avg_integral_changes.append(np.mean(step_changes))
        std_integral_changes.append(np.std(step_changes, ddof=1))
        step_max_diffs = [seed_diffs[step] for seed_diffs in all_seeds_max_diffs]
        step_rmses = [seed_rmses[step] for seed_rmses in all_seeds_rmses]
        avg_max_diffs.append(np.mean(step_max_diffs))
        avg_rmses.append(np.mean(step_rmses))
    
    return avg_integral_changes, std_integral_changes, avg_max_diffs, avg_rmses


def llm_multi_predictions_with_conservation_smaller_model(
    full_serialized_data, input_time_steps, number_of_future_predictions,
    model, tokenizer, Nx, settings, vmin, vmax, n_seeds, initial_integral,
    max_retries=10
):
    """Modified LLM prediction function with in-loop retry for smaller models that also tracks energy conservation"""
    all_rows_scaled = deserialize_2d_integers(full_serialized_data, settings)
    all_seeds_integral_changes = []
    all_seeds_max_diffs = []
    all_seeds_rmses = []
    for seed in range(n_seeds):
        if n_seeds > 1:
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        # Initialize for this seed
        train_serial, _ = extract_training_and_test(full_serialized_data, input_time_steps, settings)
        if not train_serial.endswith(settings.time_sep):
            train_serial += settings.time_sep
        current_prompt = train_serial
        integral_changes = []
        max_diffs = []
        rmses = []
        for step_idx in range(number_of_future_predictions):
            gt_idx = input_time_steps + step_idx
            if gt_idx >= all_rows_scaled.shape[0]:
                break
            valid_step = False
            for attempt in range(max_retries):
                next_line, _ = generate_text_multiple(current_prompt, model, tokenizer, Nx)
                next_line = next_line.strip()
                pred_scaled_2d = deserialize_2d_integers(next_line, settings)
                if pred_scaled_2d.shape == (1, Nx):
                    valid_step = True
                    break
                else:
                    print(f"  step {step_idx} attempt {attempt} wrong shape {pred_scaled_2d.shape}, retrying...")
            if not valid_step:
                print(f"  Warning: step {step_idx} failed all {max_retries} retries; using last output")
            predicted_unscaled_2d = unscale_2d_array(pred_scaled_2d, vmin, vmax)
            if predicted_unscaled_2d.ndim == 2 and predicted_unscaled_2d.shape[0] == 1:
                pred_unscaled = predicted_unscaled_2d[0]
            else:
                pred_unscaled = predicted_unscaled_2d
            # Compute integral and relative change
            pred_integral = compute_integral_with_boundaries(pred_unscaled, Nx, L)
            relative_change = np.abs((pred_integral - initial_integral) / np.abs(initial_integral) * 100)
            integral_changes.append(relative_change)
            # Compute error metrics
            gt_scaled = all_rows_scaled[gt_idx]
            gt_unscaled = unscale_2d_array(gt_scaled[np.newaxis, :], vmin, vmax)[0]
            max_diff = np.max(np.abs(pred_unscaled - gt_unscaled))
            rmse = np.sqrt(np.mean((pred_unscaled - gt_unscaled)**2))
            max_diffs.append(max_diff)
            rmses.append(rmse)
            current_prompt += next_line + settings.time_sep
        all_seeds_integral_changes.append(integral_changes)
        all_seeds_max_diffs.append(max_diffs)
        all_seeds_rmses.append(rmses)
    max_steps = min(len(changes) for changes in all_seeds_integral_changes)
    avg_integral_changes = []
    std_integral_changes = []
    avg_max_diffs = []
    avg_rmses = []
    for step in range(max_steps):
        step_changes = [seed_changes[step] for seed_changes in all_seeds_integral_changes]
        avg_integral_changes.append(np.mean(step_changes))
        std_integral_changes.append(np.std(step_changes, ddof=1))
        step_max_diffs = [seed_diffs[step] for seed_diffs in all_seeds_max_diffs]
        step_rmses = [seed_rmses[step] for seed_rmses in all_seeds_rmses]
        avg_max_diffs.append(np.mean(step_max_diffs))
        avg_rmses.append(np.mean(step_rmses))
    
    return avg_integral_changes, std_integral_changes, avg_max_diffs, avg_rmses


def finite_difference_predictions_with_conservation(
    full_serialized_data, input_time_steps, number_of_future_predictions,
    settings, vmin, vmax, L, k, Nt, Nx, T, initial_integral
):
    """Modified FD prediction function that also tracks energy conservation"""
    # Extract full solution from serialized data
    all_rows_scaled = deserialize_2d_integers(full_serialized_data, settings)
    dt = T / Nt
    ftcs_integral_changes = []
    btcs_integral_changes = []
    ftcs_max_diffs = []
    btcs_max_diffs = []
    ftcs_rmses = []
    btcs_rmses = []
    # Get initial condition from last training step
    initial_step = input_time_steps - 1
    initial_scaled = all_rows_scaled[initial_step]
    initial_unscaled = unscale_2d_array(initial_scaled[np.newaxis, :], vmin, vmax)[0]
    current_ftcs = initial_unscaled.copy()
    current_btcs = initial_unscaled.copy()
    for step_idx in range(number_of_future_predictions):
        gt_idx = input_time_steps + step_idx
        if gt_idx >= all_rows_scaled.shape[0]:
            # Stop if we exceed the available ground truth
            break
        # Get ground truth for this step
        gt_scaled = all_rows_scaled[gt_idx]
        gt_unscaled = unscale_2d_array(gt_scaled[np.newaxis, :], vmin, vmax)[0]
        # We set T=dt and Nt=1 to evolve exactly one time step
        _, ftcs_step, _ = solve_heat_ftcs(L, k, dt, Nx, 1, init_cond=current_ftcs)
        _, btcs_step, _ = solve_heat_btcs(L, k, dt, Nx, 1, init_cond=current_btcs)
        # Extract predictions (using last time step)
        pred_ftcs = ftcs_step[-1]
        pred_btcs = btcs_step[-1]
        # Compute integrals and relative changes
        ftcs_integral = compute_integral_with_boundaries(pred_ftcs, Nx, L)
        btcs_integral = compute_integral_with_boundaries(pred_btcs, Nx, L)
        ftcs_relative_change = np.abs((ftcs_integral - initial_integral) / np.abs(initial_integral) * 100)
        btcs_relative_change = np.abs((btcs_integral - initial_integral) / np.abs(initial_integral) * 100)
        ftcs_integral_changes.append(ftcs_relative_change)
        btcs_integral_changes.append(btcs_relative_change)
        # Compute error metrics
        ftcs_max_diffs.append(np.max(np.abs(pred_ftcs - gt_unscaled)))
        ftcs_rmses.append(np.sqrt(np.mean((pred_ftcs - gt_unscaled)**2)))
        btcs_max_diffs.append(np.max(np.abs(pred_btcs - gt_unscaled)))
        btcs_rmses.append(np.sqrt(np.mean((pred_btcs - gt_unscaled)**2)))
        # Update current state for next step
        current_ftcs = pred_ftcs.copy()
        current_btcs = pred_btcs.copy()
    
    return {
        'ftcs': {
            'integral_changes': ftcs_integral_changes,
            'max_diff': ftcs_max_diffs,
            'rmse': ftcs_rmses
        },
        'btcs': {
            'integral_changes': btcs_integral_changes,
            'max_diff': btcs_max_diffs,
            'rmse': btcs_rmses
        }
    }

In [ ]:
# Generate all random initial conditions and spline objects
stored_initial_conditions = []
stored_spline_objects = []
stored_initial_integrals_fine = []
stored_initial_integrals_coarse = []
for ic_seed in range(n_ics):
    # Set seed for this initial condition
    random.seed(ic_seed)
    np.random.seed(ic_seed)
    torch.manual_seed(ic_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(ic_seed)
    # Generate and store the random initial condition
    init_cond_random = np.random.uniform(0.0, 1.0, size=Nx)
    stored_initial_conditions.append(init_cond_random.copy())
    # Create and store the spline object
    fig, cs = visualize_spline_ic(L, Nx, init_cond_random)
    plt.close(fig)
    stored_spline_objects.append(cs)
    # Compute initial integral from fine spline grid
    x_fine_init = np.linspace(-L/2, L/2, 1000)
    u_fine_init = cs(x_fine_init)
    initial_integral_fine = np.trapezoid(u_fine_init, x_fine_init)
    stored_initial_integrals_fine.append(initial_integral_fine)
    # Compute initial integral on coarse grid for reference
    initial_integral_coarse = compute_integral_with_boundaries(init_cond_random, Nx, L)
    stored_initial_integrals_coarse.append(initial_integral_coarse)
stored_initial_conditions_array = np.array(stored_initial_conditions)


all_llm_integral_changes = []
all_llm_max_diffs = []
all_llm_rmses = []
all_fd_results = []
all_exact_integral_changes = []
for ic_seed in tqdm(range(n_ics)):
    # Use the stored initial condition, spline, and initial integral
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    initial_integral_fine = stored_initial_integrals_fine[ic_seed]
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    exact_integral_changes = compute_exact_solution_conservation(
        u_exact[input_time_steps:input_time_steps+number_of_future_predictions],
        Nx, L, initial_integral_fine
    )
    all_exact_integral_changes.append(exact_integral_changes)
    # Run LLM predictions with conservation tracking
    llm_integral_changes, llm_integral_changes_std, llm_max_diffs, llm_rmses = llm_multi_predictions_with_conservation(
        full_serialized_data=u_exact_serialized,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        model=model,
        tokenizer=tokenizer,
        Nx=Nx,
        settings=settings,
        vmin=vmin_exact,
        vmax=vmax_exact,
        n_seeds=n_runs_per_ic,
        initial_integral=initial_integral_fine
    )
    # Run finite difference predictions with conservation tracking
    fd_results = finite_difference_predictions_with_conservation(
        full_serialized_data=u_exact_serialized,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        settings=settings,
        vmin=vmin_exact,
        vmax=vmax_exact,
        L=L,
        k=k,
        Nt=Nt,
        Nx=Nx,
        T=T,
        initial_integral=initial_integral_fine
    )
    all_llm_integral_changes.append(llm_integral_changes)
    all_llm_max_diffs.append(llm_max_diffs)
    all_llm_rmses.append(llm_rmses)
    all_fd_results.append(fd_results)

# Compute averages for conservation errors
avg_llm_integral_changes = np.mean(all_llm_integral_changes, axis=0)
std_llm_integral_changes = np.std(all_llm_integral_changes, axis=0, ddof=1)
ftcs_integral_changes = [res['ftcs']['integral_changes'] for res in all_fd_results]
btcs_integral_changes = [res['btcs']['integral_changes'] for res in all_fd_results]
avg_ftcs_integral_changes = np.mean(ftcs_integral_changes, axis=0)
avg_btcs_integral_changes = np.mean(btcs_integral_changes, axis=0)
std_ftcs_integral_changes = np.std(ftcs_integral_changes, axis=0, ddof=1)
std_btcs_integral_changes = np.std(btcs_integral_changes, axis=0, ddof=1)
avg_exact_integral_changes = np.mean(all_exact_integral_changes, axis=0)
std_exact_integral_changes = np.std(all_exact_integral_changes, axis=0, ddof=1)
# Compute averages for error metrics
avg_llm_max_diffs = np.mean(all_llm_max_diffs, axis=0)
avg_llm_rmses = np.mean(all_llm_rmses, axis=0)
std_llm_max_diffs = np.std(all_llm_max_diffs, axis=0, ddof=1)
std_llm_rmses = np.std(all_llm_rmses, axis=0, ddof=1)
ftcs_max_diffs = [res['ftcs']['max_diff'] for res in all_fd_results]
ftcs_rmses = [res['ftcs']['rmse'] for res in all_fd_results]
btcs_max_diffs = [res['btcs']['max_diff'] for res in all_fd_results]
btcs_rmses = [res['btcs']['rmse'] for res in all_fd_results]
avg_ftcs_max_diff = np.mean(ftcs_max_diffs, axis=0)
avg_ftcs_rmse = np.mean(ftcs_rmses, axis=0)
avg_btcs_max_diff = np.mean(btcs_max_diffs, axis=0)
avg_btcs_rmse = np.mean(btcs_rmses, axis=0)
# Calculate confidence intervals
t_critical = stats.t.ppf(0.975, n_ics - 1)  # 95% CI
# CI for LLM error metrics
ci_lower_max_diffs_8B = []
ci_upper_max_diffs_8B = []
ci_lower_rmses_8B = []
ci_upper_rmses_8B = []
for mean, std in zip(avg_llm_max_diffs, std_llm_max_diffs):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_max_diffs_8B.append(lower)
    ci_upper_max_diffs_8B.append(upper)
for mean, std in zip(avg_llm_rmses, std_llm_rmses):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_rmses_8B.append(lower)
    ci_upper_rmses_8B.append(upper)
# CI for conservation errors
ci_lower_integral_8B = []
ci_upper_integral_8B = []
for mean, std in zip(avg_llm_integral_changes, std_llm_integral_changes):
    lower, upper = linear_ci(mean, std, n_ics, t_critical)
    ci_lower_integral_8B.append(max(lower, 0))
    ci_upper_integral_8B.append(upper)

# Compute quant floor 
all_baseline_max_errors_per_step = []
all_baseline_rmse_errors_per_step = []
for ic_seed in range(n_ics):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    # Quantization pipeline
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    u_exact_parsed = deserialize_2d_integers(u_exact_serialized, settings)
    u_exact_unscaled = unscale_2d_array(u_exact_parsed, vmin_exact, vmax_exact)
    # Compute baseline errors at each time step for this seed
    seed_max_errors_per_step = []
    seed_rmse_errors_per_step = []
    for t in range(u_exact.shape[0]):
        max_err_t = np.max(np.abs(u_exact[t] - u_exact_unscaled[t]))
        rmse_err_t = np.sqrt(np.mean((u_exact[t] - u_exact_unscaled[t])**2))
        seed_max_errors_per_step.append(max_err_t)
        seed_rmse_errors_per_step.append(rmse_err_t)
    all_baseline_max_errors_per_step.append(seed_max_errors_per_step)
    all_baseline_rmse_errors_per_step.append(seed_rmse_errors_per_step)
all_baseline_max_errors_per_step = np.array(all_baseline_max_errors_per_step)
all_baseline_rmse_errors_per_step = np.array(all_baseline_rmse_errors_per_step)
avg_baseline_max_errors_per_step = np.mean(all_baseline_max_errors_per_step, axis=0)
avg_baseline_rmse_errors_per_step = np.mean(all_baseline_rmse_errors_per_step, axis=0)
avg_baseline_max_errors_prediction = avg_baseline_max_errors_per_step[input_time_steps:]
avg_baseline_rmse_errors_prediction = avg_baseline_rmse_errors_per_step[input_time_steps:]

averaged_fd_results = {
    'ftcs': {
        'max_diff': avg_ftcs_max_diff.tolist(),
        'rmse': avg_ftcs_rmse.tolist()
    },
    'btcs': {
        'max_diff': avg_btcs_max_diff.tolist(),
        'rmse': avg_btcs_rmse.tolist()
    }
}

np.savez_compressed(
    "8B_10_step.npz",
    # LLM error metrics
    llm_max_diffs_8B=avg_llm_max_diffs,
    llm_rmses_8B=avg_llm_rmses,
    std_max_diffs_8B=std_llm_max_diffs,
    std_rmses_8B=std_llm_rmses,
    # LLM confidence intervals for error metrics
    ci_lower_max_diffs_8B=ci_lower_max_diffs_8B,
    ci_upper_max_diffs_8B=ci_upper_max_diffs_8B,
    ci_lower_rmses_8B=ci_lower_rmses_8B,
    ci_upper_rmses_8B=ci_upper_rmses_8B,
    # LLM conservation metrics
    llm_integral_changes_8B=avg_llm_integral_changes,
    std_llm_integral_changes_8B=std_llm_integral_changes,
    ci_lower_integral_8B=ci_lower_integral_8B,
    ci_upper_integral_8B=ci_upper_integral_8B,
    # FD conservation metrics
    ftcs_integral_changes=avg_ftcs_integral_changes,
    std_ftcs_integral_changes=std_ftcs_integral_changes,
    btcs_integral_changes=avg_btcs_integral_changes,
    std_btcs_integral_changes=std_btcs_integral_changes,
    # Exact solution conservation (baseline)
    exact_integral_changes=avg_exact_integral_changes,
    std_exact_integral_changes=std_exact_integral_changes,
    # FD error metrics
    ftcs_max_diffs=avg_ftcs_max_diff,
    ftcs_rmses=avg_ftcs_rmse,
    btcs_max_diffs=avg_btcs_max_diff,
    btcs_rmses=avg_btcs_rmse,
    fd_results=averaged_fd_results,
    # Quantization floor for prediction steps only
    avg_baseline_max_errors_prediction=avg_baseline_max_errors_prediction,
    avg_baseline_rmse_errors_prediction=avg_baseline_rmse_errors_prediction,
    # Raw results for all initial conditions
    all_llm_max_diffs=all_llm_max_diffs,
    all_llm_rmses=all_llm_rmses,
    all_llm_integral_changes_8B=all_llm_integral_changes,
    all_fd_results=all_fd_results,
    all_exact_integral_changes=all_exact_integral_changes,
    stored_initial_conditions=stored_initial_conditions_array,
    stored_initial_integrals_fine=stored_initial_integrals_fine,
    stored_initial_integrals_coarse=stored_initial_integrals_coarse
)